<a href="https://colab.research.google.com/github/GautierBlandin/deep-learning-notebooks/blob/main/MNIST_second_take.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import tensorflow as tf
import tensorflow.keras as keras

(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

In [15]:
x_train = x_train.reshape([x_train.shape[0], 28, 28, 1])
x_test = x_test.reshape([x_test.shape[0], 28, 28, 1])

In [3]:
# specify the width and height shift arguments
width_shift_val = 0.2
height_shift_val = 0.2

# import relevant library
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    featurewise_std_normalization=True,
    featurewise_center = True,
    rotation_range=10,
    validation_split = 0.1,
    width_shift_range=width_shift_val, 
    height_shift_range=height_shift_val
    )

datagen.fit(x_train)

train_generator = datagen.flow(x_train, y_train, batch_size=32, subset='training')
validation_generator = datagen.flow(x_train, y_train, batch_size=32, subset='validation')

In [10]:
from sklearn.preprocessing import StandardScaler 

scaler = StandardScaler()

x_test = scaler.fit_transform(x_test.reshape([len(x_test), 28 * 28])).reshape([len(x_test), 28, 28, 1])

In [17]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    featurewise_std_normalization=True,
    featurewise_center = True
    )

datagen.fit(x_test)

test_generator = datagen.flow(x_test, y_test, batch_size = 32)

In [4]:
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=[28, 28, 1]))
model.add(keras.layers.Dense(500, activation = "selu", kernel_initializer='lecun_normal'))
model.add(keras.layers.Dense(300, activation = "selu", kernel_initializer='lecun_normal'))
model.add(keras.layers.Dense(200, activation = "selu", kernel_initializer='lecun_normal'))
model.add(keras.layers.Dense(200, activation = "selu", kernel_initializer='lecun_normal'))
model.add(keras.layers.Dense(100, activation = "selu", kernel_initializer='lecun_normal'))
model.add(keras.layers.Dense(100, activation = "selu", kernel_initializer='lecun_normal'))
model.add(keras.layers.Dense(10, activation = "softmax"))

In [5]:
# Compile the model to specify the loss function and the optimizer to use.

optimizer = tf.keras.optimizers.Nadam(
    learning_rate=0.001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    name='Nadam'
)

model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])

In [6]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience = 20)
checkpoint_cb = keras.callbacks.ModelCheckpoint("mnist_model.h5", save_best_only=True)

history = model.fit(train_generator, validation_data = validation_generator, batch_size = 32, epochs=200, callbacks=[checkpoint_cb, early_stopping_cb])

Epoch 1/200
1688/1688 [==============================] - 47s 27ms/step - loss: 0.7104 - accuracy: 0.7709 - val_loss: 0.4565 - val_accuracy: 0.8632
Epoch 2/200
1688/1688 [==============================] - 43s 26ms/step - loss: 0.4044 - accuracy: 0.8726 - val_loss: 0.3419 - val_accuracy: 0.8938
Epoch 3/200
1688/1688 [==============================] - 44s 26ms/step - loss: 0.3283 - accuracy: 0.8978 - val_loss: 0.2620 - val_accuracy: 0.9222
Epoch 4/200
1688/1688 [==============================] - 44s 26ms/step - loss: 0.2916 - accuracy: 0.9101 - val_loss: 0.2489 - val_accuracy: 0.9238
Epoch 5/200
1688/1688 [==============================] - 45s 26ms/step - loss: 0.2665 - accuracy: 0.9191 - val_loss: 0.2334 - val_accuracy: 0.9302
Epoch 6/200
1688/1688 [==============================] - 43s 26ms/step - loss: 0.2438 - accuracy: 0.9266 - val_loss: 0.2419 - val_accuracy: 0.9263
Epoch 7/200
1688/1688 [==============================] - 43s 26ms/step - loss: 0.2256 - accuracy: 0.9314 - val_loss: 0

In [18]:
model = keras.models.load_model("mnist_model.h5")
model.evaluate(test_generator)

313/313 [==============================] - 2s 5ms/step - loss: 0.0657 - accuracy: 0.9828


[0.06573978811502457, 0.9828000068664551]

In [19]:
from google.colab import files
files.download('mnist_model.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>